In [38]:
import boto3
import os
from random import random
from datetime import datetime
import json

#use argparse to make final product a cli tool

def generate_templates(save_folder='recipes/template'):
    """
    Create templates needed to upload a recipe to s3
    """
    os.mkdir(save_folder)
    
    #recipe.json
    template = {
        'Title':''
        ,'Description':''
        ,'Ingredients':['']
        ,'Tags':['']
    }
    
    with open(save_folder+"/recipe.json",'w') as template_file:
        json.dump(template,template_file,indent=2)   
    
    #instructions.html
    template = """<h4>Header4</h4>
<p>This is some details</p>
<ol>
<li>First step</li>
<li>Second step</li>
</ol>"""
    
    with open(save_folder+"/instructions.html",'w') as template_file:
        template_file.write(template)

    #payload.json
    template = {
                    "instruction":"instructions.html",
                    "recipe":"recipe.json",
                    "photo":"photo.jpg"
                }
    
    with open(save_folder+"/payload.json",'w') as template_file:
        json.dump(template,template_file,indent=2)
    
    return None

def valid_recipe_image(image_file):
    """
    parameter:
    image_file (string) - path to image file for recipe
    
    response (boolean):
    True - image_file references a valid image


    """
    #only accept jpg or jpeg
    return True

def valid_recipe_json(json_file):
    #must have at least one tag
    #tags should be capitalized
    return True

def upload_image_to_s3(recipe_id,image_file,s3_client,s3_bucket,s3_image_folder):
    with open(image_file,'rb') as image:
        response = s3_client.put_object(
            Body=image,
            Bucket=s3_bucket,
            Key=f'{s3_image_folder}/{recipe_id}'
        )
    return response

def upload_json_to_s3(recipe_id,json_file,instruction_file,s3_client,s3_bucket,s3_region,s3_json_folder,s3_image_folder):
    with open(json_file,'r') as json_file:
        recipe_json = json.load(json_file)
    with open(instruction_file,'r') as instruction_file:
        instructions = instruction_file.read()
    recipe_json['Instructions'] = instructions
    recipe_json['ID'] = int(recipe_id)
    recipe_json['Photo'] = f'https://{s3_bucket}.s3-{s3_region}.amazonaws.com/{s3_image_folder}/{recipe_id}'
    response = s3_client.put_object(
        Body=json.dumps(recipe_json).encode('UTF-8'),
        Bucket=s3_bucket,
        Key=f'{s3_json_folder}/{recipe_id}.json'
    )
    return response

def upload_recipe_to_s3(image_file,json_file,instruction_file,s3_client,s3_bucket,s3_region,s3_json_folder,s3_image_folder):
    """
    Upload recipe payload to S3 for RecipeBlog
    
    Input:
        image_file (string): Path to recipe image file
        json_file (string): Path to recipe json file
        
    output (json):
        {upload_status: success|json_upload_failed|image_upload_failed|invalid_recipe_json|invalid_recpie_image}
    """
    timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
    recipe_id = timestamp+str(int(random()*10**10))
    
    #Validate Payload
    if valid_recipe_image(image_file): 
        if valid_recipe_json(json_file):
            image_upload_response = upload_image_to_s3(recipe_id,image_file,s3_client,s3_bucket,s3_image_folder)
            
            #Starting Upload
            if image_upload_response: #response 200
                json_upload_response = upload_json_to_s3(recipe_id,json_file,instruction_file,s3_client,s3_bucket,s3_region,s3_json_folder,s3_image_folder)
                if json_upload_response: #response 200
                    #Successful Upload
                    return {'upload_status':'success'}
                
                #Unsuccessful Uploads
                else:
                    return {'upload_status':'json_upload_failed'}
            else:
                return {'upload_status':'image_upload_failed'}
        else:
            return {'upload_status':'invalid_recipe_json'}
    else:
        return {'upload_status':'invalid_recpie_image'}
    return None

def upload_recipe_from_payload(bucket_info,recipe_directory):
    with open(recipe_directory+'/payload.json','r') as payload:
        payload = json.load(payload)
        
    image_file = recipe_directory+'/'+payload['photo']
    json_file = recipe_directory+'/'+payload['recipe']
    instruction_file = recipe_directory+'/'+payload['instruction']

    s3_client = boto3.session.Session().client('s3')
    s3_bucket = bucket_info['s3_bucket']
    s3_region = bucket_info['s3_region']
    s3_image_folder = bucket_info['s3_image_folder']
    s3_json_folder = bucket_info['s3_json_folder']


    response = upload_recipe_to_s3(image_file,json_file,instruction_file
                                   ,s3_client,s3_bucket,s3_region
                                   ,s3_json_folder,s3_image_folder
                                  )
    return response

In [39]:
#generate_templates(save_folder='recipes/template')


In [40]:
with open('./bucket_info.json','r') as bucket_info:
    bucket_info = json.load(bucket_info)

In [41]:
upload_recipe_from_payload(bucket_info = bucket_info, recipe_directory = 'recipes/beef_noodle_soup')

{'upload_status': 'success'}